In [95]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

import numpy as np
import pandas as pd

In [96]:
def create_padding_mask(seq):
    # 패딩 처리된 부분을 마스킹합니다.
    # 모델이 패딩을 입력으로 취급하지 않도록 
    seq = tf.cast(tf.math.equal(seq, 0), tf.float32)

    return seq[:, tf.newaxis, tf.newaxis, :]  # (batch_size, 1, 1, seq_len)

x = tf.constant([[7, 6, 5, 4, 0], [1, 2, 3, 0, 0], [1, 8, 0, 0, 0]])
create_padding_mask(x)

<tf.Tensor: shape=(3, 1, 1, 5), dtype=float32, numpy=
array([[[[0., 0., 0., 0., 1.]]],


       [[[0., 0., 0., 1., 1.]]],


       [[[0., 0., 1., 1., 1.]]]], dtype=float32)>

In [97]:
def get_angles(pos, i, d_model):
    """
    sin, cos 안에 들어갈 수치를 구하는 함수입니다.
    """
    angle_rates = 1 / np.power(10000, (2 * (i//2)) / np.float32(d_model))
    return pos * angle_rates

def positional_encoding(position, d_model):
    """
    위치 인코딩(Positional Encoding)을 구하는 함수입니다.
    """
    angle_rads = get_angles(np.arange(position)[:, np.newaxis],
                          np.arange(d_model)[np.newaxis, :],
                          d_model)

    # apply sin to even indices in the array; 2i
    angle_rads[:, 0::2] = np.sin(angle_rads[:, 0::2])

    # apply cos to odd indices in the array; 2i+1
    angle_rads[:, 1::2] = np.cos(angle_rads[:, 1::2])

    pos_encoding = angle_rads[np.newaxis, ...]

    return tf.cast(pos_encoding, dtype=tf.float32)

In [98]:
def scaled_dot_product_attention(q, k, v, mask):
    
    """Calculate the attention weights.
    q, k, v must have matching leading dimensions.
    k, v must have matching penultimate dimension, i.e.: seq_len_k = seq_len_v.
    The mask has different shapes depending on its type(padding or look ahead) 
    but it must be broadcastable for addition.
  
    Args:
        q: query shape == (..., seq_len_q, depth)
        k: key shape == (..., seq_len_k, depth)
        v: value shape == (..., seq_len_v, depth_v)
        mask: Float tensor with shape broadcastable 
            to (..., seq_len_q, seq_len_k). Defaults to None.
    
    Returns:
        output, attention_weights
    """
    
    matmul_qk = tf.matmul(q, k, transpose_b=True)  # (..., seq_len_q, seq_len_k)
    
    # scale matmul_qk
    dk = tf.cast(tf.shape(k)[-1], tf.float32)
    scaled_attention_logits = matmul_qk / tf.math.sqrt(dk)

    """
    mask가 있을 경우 masking된 자리(mask=1)에는 (-inf)에 해당하는 절댓값이 큰 음수 -1e9(=-10억)을 더해줍니다.
    그 값에 softmax를 취해주면 거의 0에 가까운 값이 나옵니다. 그 다음 value 계산시에 반영되지 않습니다.
    """
    
    # add the mask to the scaled tensor.
    if mask is not None:
        scaled_attention_logits += (mask * -1e9)  
        
    # softmax is normalized on the last axis (seq_len_k) so that the scores
    # add up to 1.
    attention_weights = tf.nn.softmax(scaled_attention_logits, axis=-1)  # (..., seq_len_q, seq_len_k)
    
    output = tf.matmul(attention_weights, v)  # (..., seq_len_q, depth_v)
    
    return output, attention_weights

In [99]:
def point_wise_feed_forward_network(**kargs):
    """
    FFNN을 구현한 코드입니다.

    Args:
        d_model : 모델의 차원입니다.
        dff : 은닉층의 차원 수입니다. 논문에서는 2048을 사용하였습니다.
    """
    return tf.keras.Sequential([
      tf.keras.layers.Dense(kargs['dff'], activation='relu'),  # (batch_size, seq_len, dff)
      tf.keras.layers.Dense(kargs['d_model'])  # (batch_size, seq_len, d_model)
    ])

In [100]:
class MultiHeadAttention(tf.keras.layers.Layer):
    def __init__(self, **kargs):
        super(MultiHeadAttention, self).__init__()
        self.num_heads = kargs['num_heads']
        self.d_model = kargs['d_model']

        assert self.d_model % self.num_heads == 0

        self.depth = self.d_model // self.num_heads

        self.wq = tf.keras.layers.Dense(self.d_model)
        self.wk = tf.keras.layers.Dense(self.d_model)
        self.wv = tf.keras.layers.Dense(self.d_model)

        self.dense = tf.keras.layers.Dense(kargs['d_model'])

    def split_heads(self, x, batch_size):
        """
        Split the last dimension into (num_heads, depth).
        Transpose the result such that the shape is (batch_size, num_heads, seq_len, depth)
        """
        x = tf.reshape(x, (batch_size, -1, self.num_heads, self.depth))
        return tf.transpose(x, perm=[0, 2, 1, 3])

    def call(self, v, k, q, mask):
        batch_size = tf.shape(q)[0]

        q = self.wq(q)  # (batch_size, seq_len, d_model)
        k = self.wk(k)  # (batch_size, seq_len, d_model)
        v = self.wv(v)  # (batch_size, seq_len, d_model)

        q = self.split_heads(q, batch_size)  # (batch_size, num_heads, seq_len_q, depth)
        k = self.split_heads(k, batch_size)  # (batch_size, num_heads, seq_len_k, depth)
        v = self.split_heads(v, batch_size)  # (batch_size, num_heads, seq_len_v, depth)

        # scaled_attention.shape == (batch_size, num_heads, seq_len_q, depth)
        # attention_weights.shape == (batch_size, num_heads, seq_len_q, seq_len_k)
        scaled_attention, attention_weights = scaled_dot_product_attention(
            q, k, v, mask)

        scaled_attention = tf.transpose(scaled_attention, perm=[0, 2, 1, 3])  # (batch_size, seq_len_q, num_heads, depth)

        concat_attention = tf.reshape(scaled_attention, 
                                      (batch_size, -1, self.d_model))  # (batch_size, seq_len_q, d_model)

        output = self.dense(concat_attention)  # (batch_size, seq_len_q, d_model)

        return output, attention_weights

In [101]:
# 인코더 레이어
class EncoderLayer(tf.keras.layers.Layer):
    def __init__(self, **kargs):
        super(EncoderLayer, self).__init__()

        self.mha = MultiHeadAttention(**kargs)
        self.ffn = point_wise_feed_forward_network(**kargs)

        self.layernorm1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)

        self.dropout1 = tf.keras.layers.Dropout(kargs['rate'])
        self.dropout2 = tf.keras.layers.Dropout(kargs['rate'])

    def call(self, x, mask):
        attn_output, _ = self.mha(x, x, x, mask)  # (batch_size, input_seq_len, d_model)
        attn_output = self.dropout1(attn_output)
        out1 = self.layernorm1(x + attn_output)  # (batch_size, input_seq_len, d_model)

        ffn_output = self.ffn(out1)  # (batch_size, input_seq_len, d_model)
        ffn_output = self.dropout2(ffn_output)
        out2 = self.layernorm2(out1 + ffn_output)  # (batch_size, input_seq_len, d_model)

        return out2

In [102]:
# 인코더
class Encoder(tf.keras.layers.Layer):
    def __init__(self, **kwargs):
        super(Encoder, self).__init__()

        self.d_model = kwargs['d_model']
        self.num_layers = kwargs['num_layers']

        self.embedding = tf.keras.layers.Embedding(kwargs['vocab_size'], self.d_model)
        self.pos_encoding = positional_encoding(kwargs['maximum_position_encoding'], 
                                                self.d_model)


        self.enc_layers = [EncoderLayer(**kwargs) 
                           for _ in range(self.num_layers)]

        self.dropout = tf.keras.layers.Dropout(kwargs['rate'])

    def call(self, x, mask):

        seq_len = tf.shape(x)[1]

        # adding embedding and position encoding.
        x = self.embedding(x)  # (batch_size, input_seq_len, d_model)
        x *= tf.math.sqrt(tf.cast(self.d_model, tf.float32))
        x += self.pos_encoding[:, :seq_len, :]

        x = self.dropout(x)

        for i in range(self.num_layers):
            x = self.enc_layers[i](x, mask)

        return x  # (batch_size, input_seq_len, d_model)

In [103]:
class Transformer(tf.keras.Model):
    def __init__(self, **kargs):
        super(Transformer, self).__init__()
        
        self.encoder = Encoder(**kargs)
        self.classification_layer = tf.keras.layers.Dense(1, activation='sigmoid')

    def call(self, inputs, training):
        # Keras models prefer if you pass all your inputs in the first argument

        enc_padding_mask = self.create_masks(inputs)

        enc_output = self.encoder(inputs, enc_padding_mask)  # (batch_size, inp_seq_len, d_model)

        final_output = self.classification_layer(enc_output[:, 0, :]) # (bs, 1, d_model)

        return final_output

    def create_masks(self, inputs):
        # Encoder padding mask
        enc_padding_mask = create_padding_mask(inputs)

        return enc_padding_mask

In [104]:
BATCH_SIZE = 64
MAX_SEQUENCE = 80
EPOCHS = 3
VALID_SPLIT = 0.1
MAX_FEATURES = 20000
MAXLEN = 2000

kargs = {'num_layers': 2,
         'd_model': 512,
         'num_heads': 8,
         'dff': 2048,
         'vocab_size': MAX_FEATURES,
         'maximum_position_encoding': MAXLEN,
         'rate': 0.1
        }

In [105]:
df = pd.read_csv('moviereviews.tsv', sep='\t')

In [106]:
df = df.dropna()
df.drop_duplicates(subset=['review'], inplace=True)

In [107]:
import re

for idx, text in enumerate(df['review']):
    text = text.lower()
    text = text.replace('\n', ' ')
    text = re.sub(r'[^a-z0-9 ]', '', text)
    df['review'][idx] = text

In [108]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

X = df['review']
Y = df['label']

le = LabelEncoder()
Y = le.fit_transform(Y)
Y = Y.reshape(-1,1)

# 데이터셋 분리
X_train, X_test, Y_train, Y_test = train_test_split(
    X, Y, test_size=0.2, stratify=df['label'], random_state=42)

In [109]:
#토큰화
from tensorflow.keras.preprocessing.text import Tokenizer

max_words = 3000
max_len = 6000
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(X_train)
train_sequences = tokenizer.texts_to_sequences(X_train)
test_sequences = tokenizer.texts_to_sequences(X_test)

In [110]:
X_train_pad = tf.keras.preprocessing.sequence.pad_sequences(train_sequences, maxlen=MAXLEN)
X_test_pad = tf.keras.preprocessing.sequence.pad_sequences(test_sequences, maxlen=MAXLEN)

In [111]:
#모델 선언
model = Transformer(**kargs)
model.compile(optimizer=tf.keras.optimizers.Adam(1e-4),
              loss='binary_crossentropy',
              metrics='accuracy')

In [112]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

earlystop_callback = EarlyStopping(monitor='val_accuracy', min_delta=0.0001, patience=10)

#학습시킨 데이터 저장
checkpoint_path = 'weights.h5'

cp_callback = ModelCheckpoint(
    checkpoint_path, monitor='val_accuracy', verbose=1, save_best_only=True, save_weights_only=True)

In [113]:
# 모델 학습
history = model.fit(X_train_pad, Y_train, 
                    batch_size=BATCH_SIZE, epochs=EPOCHS,
                    verbose=1,
                    validation_split=VALID_SPLIT, callbacks=[earlystop_callback, cp_callback])

Epoch 1/3
22/22 [==============================] - ETA: 0s - loss: 1.0828 - accuracy: 0.4860  
Epoch 1: val_accuracy improved from -inf to 0.56410, saving model to weights.h5
22/22 [==============================] - 1385s 63s/step - loss: 1.0828 - accuracy: 0.4860 - val_loss: 0.6942 - val_accuracy: 0.5641
Epoch 2/3
22/22 [==============================] - ETA: 0s - loss: 0.7247 - accuracy: 0.5075  
Epoch 2: val_accuracy did not improve from 0.56410
22/22 [==============================] - 1534s 70s/step - loss: 0.7247 - accuracy: 0.5075 - val_loss: 0.6910 - val_accuracy: 0.5577
Epoch 3/3
22/22 [==============================] - ETA: 0s - loss: 0.6967 - accuracy: 0.5391  
Epoch 3: val_accuracy did not improve from 0.56410
22/22 [==============================] - 1477s 68s/step - loss: 0.6967 - accuracy: 0.5391 - val_loss: 0.7022 - val_accuracy: 0.4551


In [115]:
# 학습된 모델을 이용하여 테스트
test_loss, test_acc = model.evaluate(X_test_pad,  Y_test, verbose=1)

13/13 [==============================] - 100s 8s/step - loss: 0.6972 - accuracy: 0.4820


In [117]:
model.load_weights(checkpoint_path)

# best model을 이용한 테스트 데이터 예측 정확도 재확인
test_loss, test_acc = model.evaluate(X_test_pad, Y_test, verbose=1)

13/13 [==============================] - 101s 8s/step - loss: 0.7275 - accuracy: 0.5000
